# 第5章 一等对象——函数
一等对象的特征:
* 在运行时创建
* 能赋值给变量或数据结构中的元素
* 能作为参数传给函数
* 能作为函数的返回结果

## 5.1 把函数视作对象
Python函数是对象。这里我们创建了一个函数, 然后调用它, 读取它的\_\_doc\_\_属性

In [11]:
from functools import reduce
def factorial(n):
    """returns n!"""
    if n < 1:
        return 1
    return reduce(lambda x, y: x*y, range(1, n+1))

In [12]:
factorial(3)

6

In [13]:
factorial.__doc__

'returns n!'

In [17]:
factorial.__call__(4)

24

## 5.2 高阶函数
接收函数作为参数, 或者是返回函数的就是高阶函数

主要的三个匿名函数: map, filter, reduce

In [14]:
fruits = ["star", "a", "av"]
sorted(fruits, key=len)

['a', 'av', 'star']

## 5.3 匿名函数
lambda关键字在Python表达式内创建匿名函数

## 5.4 可调用对象
除了用户定义的函数, 调用运算符还可以应用到其他对象上。

In [19]:
callable(factorial)  # 判断对象是否可以调用

True

## 5.5 用户定义的可调用类型

In [22]:
import random

class BingoCage(object):
    
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError("pick from empty BingoCage")
            
    def __call__(self):
        return self.pick()

In [23]:
bingo = BingoCage(range(10))

In [24]:
bingo()

5

In [25]:
bingo()

4

In [26]:
callable(bingo)

True

## 5.6 函数内省

In [27]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [30]:
# 下面看看函数有而对象没有的属性
class C:pass
def func():pass

In [36]:
set(dir(func)) - set(dir(C))

{'__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__'}

## 5.7 从定位参数到仅限关键字参数
常规参数出现在可变参数之后, 这个时候就只能使用key才能传入这个常规参数

In [64]:
def tag(name = "lsl", k = 22, *context, key=11, **kwargs):
    a = 1
    pass

In [65]:
tag.__defaults__  # 这里保存着定位参数和关键字参数的默认值, key这里是仅限关键字参数

('lsl', 22)

In [66]:
tag.__kwdefaults__  # 保存着仅限关键字参数的默认值

{'key': 11}

In [67]:
print(tag.__code__.co_varnames)  # 保存函数的参数名字, 但是这里面还有函数定义的局部变量

('name', 'k', 'key', 'context', 'kwargs', 'a')


In [68]:
tag.__code__.co_argcount

2

## 5.7 获取参数信息
上面使用函数对象的magic attr的方式提取参数比较麻烦, 所以需要用到一个库来简化操作

In [69]:
import inspect

In [80]:
sig = inspect.signature(tag)
sig.parameters

mappingproxy({'name': <Parameter "name='lsl'">,
              'k': <Parameter "k=22">,
              'context': <Parameter "*context">,
              'key': <Parameter "key=11">,
              'kwargs': <Parameter "**kwargs">})

In [72]:
my_tag = {"name": "img", "k": 11, "src":"fff"}
bound_args = sig.bind(**my_tag)

In [74]:
for name, value in bound_args.arguments.items():
    print(name, "=", value)

name = img
k = 11
kwargs = {'src': 'fff'}


## 5.9 函数注解
Python3提供了一种句法, 用于为函数声明参数和返回值

In [82]:
def clip(text:str, max_len:'int > 0'=80) -> str:
    """
        裁剪文本
    """
    clip_len = len(text) if max_len > len(text) else max_len
    print("clipping")
    return text[:clip_len]

函数中的各个参数可以在:之后增加注解表达式。注解中最常用的类型是类和字符串。

In [84]:
clip.__annotations__  # 注解会被保存到这里, Python解释器并不会对注解进行检查和验证

{'text': str, 'max_len': 'int > 0', 'return': str}

In [93]:
clip(1, "2")  # 并不会检查类型

TypeError: object of type 'int' has no len()

In [85]:
clip_sig = inspect.signature(clip)

In [92]:
for key in clip_sig.parameters:
    print("==========================")
    param = clip_sig.parameters[key]
    print(param, "anno: ", param.annotation)

text:str anno:  <class 'str'>
max_len:'int > 0'=80 anno:  int > 0


## 5.10 支持函数式编程的包

### 5.10.1 operator模块

In [94]:
from functools import reduce

In [95]:
# 看一个普通阶乘的例子
def fact(n):
    return reduce(lambda x, y: x*y, range(1, n+1))

In [96]:
import operator
# 为了避免写上面那种乘法的平凡匿名表达式, operator模块提供了很多的函数
def fact1(n):
    return reduce(operator.mul, range(1, n+1))

### 5.10.2 使用functools.partial冻结参数

In [97]:
# 看下面这个例子
from functools import partial

In [100]:
triple = partial(operator.mul, 3)  # 把mul函数第一个参数固定为3
triple(4)

12